# 作業目標:
    
    建立一個網路模型

# 作業重點:

請修改 Name 中, 自定義的 Layer 名稱

增加一層全連階層

宣告 MODEL API, 分別採用自行定義的 Input/Output Layer

model.summary 查看 Layers stack

In [2]:
from keras.layers import Input, Embedding, LSTM, Dense
from keras.models import Model
import keras
#主要輸入接收新聞標題本身，即一個整數序列（每個整數編碼一個詞）。
#這些整數在1 到10,000 之間（10,000 個詞的詞彙表），且序列長度為100 個詞
#宣告一個 NAME 去定義Input
main_input = Input(shape=(100,), dtype='int32', name='Title_of_News')


# Embedding 層將輸入序列編碼為一個稠密向量的序列，
# 每個向量維度為 512。
x = Embedding(output_dim=512, input_dim=10000, input_length=100, name='Embedding')(main_input)

# LSTM 層把向量序列轉換成單個向量，
# 它包含整個序列的上下文信息
lstm_out = LSTM(32)(x)

In [3]:
#插入輔助損失，使得即使在模型主損失很高的情況下，LSTM 層和Embedding 層都能被平穩地訓練
news_output = Dense(1, activation='sigmoid', name='news_out')(lstm_out)

In [5]:
#輔助輸入數據與LSTM 層的輸出連接起來，輸入到模型
news_input = Input(shape=(5,), name='news_in')
x = keras.layers.concatenate([lstm_out, news_input])
#自行增加
news_output2 = Dense(1, activation='sigmoid', name='news_out2')(x)
news_input2 = Input(shape=(5,), name='news_in2')
x = keras.layers.concatenate([x, news_input2])

# 堆疊多個全連接網路層
x = Dense(64, activation='relu',name='HiddenLayer1')(x)
x = Dense(64, activation='relu',name='HiddenLayer2')(x)
x = Dense(64, activation='relu',name='HiddenLayer3')(x)
x = Dense(64, activation='relu',name='HiddenLayer4')(x)


# 最後添加主要的邏輯回歸層
main_output = Dense(1, activation='sigmoid', name='main_output')(x)

In [7]:
# 宣告 MODEL API, 分別採用自行定義的 Input/Output Layer
model = Model(inputs=[main_input, news_input,news_input2], outputs=[main_output, news_output,news_output2])

In [8]:
model.compile(optimizer='rmsprop',
              loss={'main_output': 'binary_crossentropy', 'news_out': 'binary_crossentropy','news_out2':'binary_crossentropy'},
              loss_weights={'main_output': 1., 'news_out': 0.2, 'news_out2':0.1})





In [9]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Title_of_News (InputLayer)      (None, 100)          0                                            
__________________________________________________________________________________________________
Embedding (Embedding)           (None, 100, 512)     5120000     Title_of_News[0][0]              
__________________________________________________________________________________________________
lstm_1 (LSTM)                   (None, 32)           69760       Embedding[0][0]                  
__________________________________________________________________________________________________
news_in (InputLayer)            (None, 5)            0                                            
____________________________________________________________________________________________